In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Amyotrophic_Lateral_Sclerosis"
cohort = "GSE212134"

# Input paths
in_trait_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis"
in_cohort_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis/GSE212134"

# Output paths
out_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/GSE212134.csv"
out_gene_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE212134.csv"
out_clinical_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE212134.csv"
json_path = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Establishing mRNA and microRNA interactions driving disease heterogeneity in Amyotrophic lateral sclerosis"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['gender: Female', 'gender: Male']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series title, it mentions "mRNA and microRNA interactions", 
# indicating gene expression data is likely available.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# There's no explicit disease/trait status in the characteristics dictionary
# The dataset is about ALS, but we don't see any classification of subjects
trait_row = None

# Age data is not present in the sample characteristics
age_row = None

# Gender is available at index 0
gender_row = 0

# 2.2 Data Type Conversion
# Since trait data is not available, we define a placeholder function
def convert_trait(value):
    return None

# Age conversion function (though not used in this case)
def convert_age(value):
    return None

# Gender conversion function
def convert_gender(value):
    if not value or ":" not in value:
        return None
    
    gender = value.split(":", 1)[1].strip().lower()
    
    if "female" in gender:
        return 0
    elif "male" in gender:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save the cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Skip the clinical feature extraction since trait_row is None (is_trait_available is False)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['2315554', '2315633', '2315674', '2315739', '2315894', '2315918',
       '2315951', '2316218', '2316245', '2316379', '2316558', '2316605',
       '2316746', '2316905', '2316953', '2317246', '2317317', '2317434',
       '2317472', '2317512'],
      dtype='object', name='ID')

Gene data dimensions: 22011 genes × 42 samples


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, these appear to be numeric probe IDs from a microarray platform
# rather than standard human gene symbols (which would typically be alphanumeric like APOE, TP53, etc.)
# These numeric IDs (like '2315554') need to be mapped to official gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // D

### Step 6: Gene Identifier Mapping

In [7]:
# Identify the column containing gene/probe IDs and gene symbols
# From the gene_annotation preview, we can see:
# - 'ID' column contains the same numeric identifiers as in the gene expression data
# - 'gene_assignment' column contains gene information with gene symbols

# 1. Identify mapping columns and create the mapping dataframe
id_col = 'ID'  # Column containing probe IDs
gene_col = 'gene_assignment'  # Column containing gene symbols

# 2. Create a mapping dataframe from gene annotation
gene_mapping = get_gene_mapping(gene_annotation, id_col, gene_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# The 'apply_gene_mapping' function handles splitting values among multiple genes
# and summing contributions for each gene
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Check the result - print dimensions and preview some gene symbols
print(f"\nAfter mapping: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("\nFirst 10 gene symbols after mapping:")
print(gene_data.index[:10])



After mapping: 48895 genes × 42 samples

First 10 gene symbols after mapping:
Index(['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the index of gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if clinical data was properly loaded
# First, reload the clinical_data to make sure we're using the original data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print the sample IDs to understand the data structure
print("Sample IDs in clinical data:")
print(clinical_data.columns[:5], "...")  # Show first 5 sample IDs

# Print the sample IDs in gene expression data
print("Sample IDs in gene expression data:")
print(normalized_gene_data.columns[:5], "...")  # Show first 5 sample IDs

# Check trait availability from previous steps
is_trait_available = trait_row is not None
linked_data = None

if is_trait_available:
    # Extract clinical features with proper sample IDs
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print(f"Clinical data preview: {preview_df(selected_clinical_df, n=3)}")
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    
    # 3. Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine if trait and demographic features are biased
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Data shape after removing biased features: {linked_data.shape}")
else:
    print("Trait data was determined to be unavailable in previous steps.")
    is_biased = True  # Dataset can't be used without trait data
    linked_data = pd.DataFrame()  # Empty DataFrame instead of artificial data

# 5. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data if not linked_data.empty else pd.DataFrame(index=normalized_gene_data.columns),
    note="Dataset contains gene expression data from ALS patients, but lacks trait information (disease status) required for associational studies."
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associational studies.")

Gene data shape after normalization: (18418, 42)
First 5 gene symbols after normalization: Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE212134.csv
Sample IDs in clinical data:
Index(['!Sample_geo_accession', 'GSM6509811', 'GSM6509812', 'GSM6509813',
       'GSM6509814'],
      dtype='object') ...
Sample IDs in gene expression data:
Index(['GSM6509811', 'GSM6509812', 'GSM6509813', 'GSM6509814', 'GSM6509815'], dtype='object') ...
Trait data was determined to be unavailable in previous steps.
Abnormality detected in the cohort: GSE212134. Preprocessing failed.
Dataset deemed not usable for associational studies.
